## Compute HCF variables not only at 12z as we were doing in HCF.ipynb, but also for 18z, 21z, and 00z. This enables us to use the retrospective method for determining if CI was triggered locally or not. 


In [2]:
# Import libraries 
import comet as cm 
import numpy as np 
import xarray as xr 
import pickle
import pandas as pd
import datetime 
import datetime 
import time 
from ComputeHCF import HCF 
from joblib import Parallel, delayed 

# Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.util


## 1. Read in data 

### 1.1 Read in data for CAM6 + CLM5

In [3]:
# Set up strings used to define/access each file 
dataDir    = '/Users/mdfowler/Documents/Analysis/Coupling_initial/data/3hrSim_CAM6-CLM5/'

fileStart  = 'f.e21.FHIST.f09_f09.cesm2_cam6_clm5.001.cam.h4.'
profileEnd = '_conusAllTimes-VertProfiles.nc'
pressEnd   = '_conus-Pressure-UTCtimes.nc'

years = np.arange(1980,1983).astype(str)

# Read in yearly files 
for iYr in range(len(years)): 
    # Set up file names 
    fileName_profile = dataDir+fileStart+years[iYr]+profileEnd
    fileName_press   = dataDir+fileStart+years[iYr]+pressEnd
    
    # Open files and save to larger arrays 
    with xr.open_dataset(fileName_profile, decode_times=True) as profileDS:
        profileDS['time'] = profileDS.indexes['time'].to_datetimeindex()
        profileDS
        
        if iYr==0:
            fullDS_profiles = profileDS
        else: 
            fullDS_profiles = xr.concat([fullDS_profiles, profileDS], dim='time')

            
    with xr.open_dataset(fileName_press, decode_times=True) as pressDS:
        pressDS['time'] = pressDS.indexes['time'].to_datetimeindex()
        
        if iYr==0:
            fullDS_pressure = pressDS
        else: 
            fullDS_pressure = xr.concat([fullDS_pressure,pressDS], dim='time')
    
    print('Done with reading in files for year %s' % years[iYr])
    

<ipython-input-3-72071a3316c3>:18: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  profileDS['time'] = profileDS.indexes['time'].to_datetimeindex()
<ipython-input-3-72071a3316c3>:28: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  pressDS['time'] = pressDS.indexes['time'].to_datetimeindex()


Done with reading in files for year 1980
Done with reading in files for year 1981
Done with reading in files for year 1982


In [4]:
ds_CLM5 = fullDS_profiles
ds_CLM5['PRESSURE'] = (('time','lev','lat','lon'), fullDS_pressure.PRESSURE)
ds_CLM5['UTC_hr'] = (('time'), fullDS_pressure.UTC_hr)
ds_CLM5['UTC_day'] = (('time'), fullDS_pressure.UTC_day)
ds_CLM5['UTC_mon'] = (('time'), fullDS_pressure.UTC_mon)
ds_CLM5['UTC_yr'] = (('time'), fullDS_pressure.UTC_yr)


### 1.2 Read in data for CAM6 + CLM4.5

In [5]:
# Set up strings used to define/access each file 
dataDir    = '/Users/mdfowler/Documents/Analysis/Coupling_initial/data/3hrSim_CAM6-CLM45/'

fileStart  = 'f.e21.FHIST.f09_f09.cesm2_cam6_clm4p5.001.cam.h4.'
profileEnd = '_conusAllTimes-VertProfiles.nc'
pressEnd   = '_conus-Pressure-UTCtimes.nc'

years = np.arange(1980,1983).astype(str)


# Read in yearly files 
for iYr in range(len(years)): 
    # Set up file names 
    fileName_profile = dataDir+fileStart+years[iYr]+profileEnd
    fileName_press   = dataDir+fileStart+years[iYr]+pressEnd
    
    # Open files and save to larger arrays 
    with xr.open_dataset(fileName_profile, decode_times=True) as profileDS:
        profileDS['time'] = profileDS.indexes['time'].to_datetimeindex()
        profileDS
        
        if iYr==0:
            fullDS_profiles = profileDS
        else: 
            fullDS_profiles = xr.concat([fullDS_profiles, profileDS], dim='time')

            
    with xr.open_dataset(fileName_press, decode_times=True) as pressDS:
        pressDS['time'] = pressDS.indexes['time'].to_datetimeindex()
        
        if iYr==0:
            fullDS_pressure = pressDS
        else: 
            fullDS_pressure = xr.concat([fullDS_pressure,pressDS], dim='time')
    
    print('Done with reading in files for year %s' % years[iYr])


<ipython-input-5-269b4c824156>:19: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  profileDS['time'] = profileDS.indexes['time'].to_datetimeindex()
<ipython-input-5-269b4c824156>:29: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  pressDS['time'] = pressDS.indexes['time'].to_datetimeindex()


Done with reading in files for year 1980
Done with reading in files for year 1981
Done with reading in files for year 1982


In [6]:
ds_CLM45 = fullDS_profiles
ds_CLM45['PRESSURE'] = (('time','lev','lat','lon'), fullDS_pressure.PRESSURE)
ds_CLM45['UTC_hr'] = (('time'), fullDS_pressure.UTC_hr)
ds_CLM45['UTC_day'] = (('time'), fullDS_pressure.UTC_day)
ds_CLM45['UTC_mon'] = (('time'), fullDS_pressure.UTC_mon)
ds_CLM45['UTC_yr'] = (('time'), fullDS_pressure.UTC_yr)


### 1.3 Isolate 18z, 21z, and 00z into their own datasets

In [7]:
# Pick out 18Z 
ds_utc18_CLM45 = ds_CLM45.where( ds_CLM45.UTC_hr==18.0, drop=True )
ds_utc18_CLM5  = ds_CLM5.where(  ds_CLM5.UTC_hr==18.0,  drop=True )
print('18Z isolated')

# Pick out 21Z 
ds_utc21_CLM45 = ds_CLM45.where( ds_CLM45.UTC_hr==21.0, drop=True )
ds_utc21_CLM5  = ds_CLM5.where(  ds_CLM5.UTC_hr==21.0,  drop=True )
print('21Z isolated')

# Pick out 00Z 
ds_utc00_CLM45 = ds_CLM45.where( ds_CLM45.UTC_hr==0.0, drop=True )
ds_utc00_CLM5  = ds_CLM5.where(  ds_CLM5.UTC_hr==0.0,  drop=True )
print('00Z isolated')


18Z isolated
21Z isolated
00Z isolated


In [8]:
# Also get lat and lon
lat = ds_utc18_CLM45.lat.values
lon = ds_utc18_CLM45.lon.values 


## 2. Compute HCF and save variables 

In [43]:
def my_HCF_parallel(iT):
    
    # Define variable names 
    Tname = 'T'
    Qname = 'Q'
    Zname = 'Z3'
    Pname = 'PRESSURE'

    TBM_all     = np.full([len(lat),len(lon)], np.nan)
    BCLH_all    = np.full([len(lat),len(lon)], np.nan)
    BCLP_all    = np.full([len(lat),len(lon)], np.nan)
    TDEF_all    = np.full([len(lat),len(lon)], np.nan)
    TRAN_H_all  = np.full([len(lat),len(lon)], np.nan)
    TRAN_P_all  = np.full([len(lat),len(lon)], np.nan)
    TRAN_T_all  = np.full([len(lat),len(lon)], np.nan)
    SHDEF_M_all = np.full([len(lat),len(lon)], np.nan)
    LHDEF_M_all = np.full([len(lat),len(lon)], np.nan)
    EADV_M_all  = np.full([len(lat),len(lon)], np.nan)


    # -------------------------------------
    #  NOTE: This section SHOULD BE MODIFIED 
    #        to change which model version/
    #        hour is desired 
    # -------------------------------------
    ds_interest = ds_utc18_CLM45
    ds_interest = ds_interest.drop('time_bnds')  # Want to get rid of nbnd dimension 

    # Want to also drop ilev dimension
    ds_interest = ds_interest.drop('hyai')
    ds_interest = ds_interest.drop('hybi') 
    ds_interest = ds_interest.drop('ilev')
    
    # Number of levels to worry about in actual "sounding"
    nLev  = len(ds_interest.lev)

    
    for iLat in range(len(lat)):
        for iLon in range(len(lon)): 
            # Pick out specific point and time period 
            DF  = ds_interest.isel(lat=iLat,lon=iLon,time=iT).to_dataframe()

            # Flip order of levels so that surface comes first (required for function)
            DF = DF.reindex(index=DF.index[::-1])

            TBM_all[iLat,iLon],BCLH_all[iLat,iLon],BCLP_all[iLat,iLon],TDEF_all[iLat,iLon],TRAN_H_all[iLat,iLon],TRAN_P_all[iLat,iLon],TRAN_T_all[iLat,iLon],SHDEF_M_all[iLat,iLon],LHDEF_M_all[iLat,iLon], EADV_M_all[iLat,iLon] = HCF(DF, 
                                                                              Tname, 
                                                                              Qname, 
                                                                              Zname, 
                                                                              Pname, 
                                                                              nLev) 
    
    return TBM_all,BCLH_all,BCLP_all,TDEF_all,TRAN_H_all,TRAN_P_all,TRAN_T_all,SHDEF_M_all,LHDEF_M_all,EADV_M_all



In [44]:
r1 = Parallel(n_jobs=2)(delayed(my_HCF_parallel)(iT) for iT in range(5))


In [45]:
np.shape(r1)


(5, 10, 43, 65)

In [46]:
TBM,BCLH,BCLP,TDEF,TRANH,TRANP,TRANT,SHDEFM,LHDEFM,EADVM = zip(*r1)



In [47]:
np.shape(TBM)



(5, 43, 65)